In [30]:
import sys
from tqdm import tqdm
import pdb
import torch
import logging
import numpy as np
import dgl
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split
import random

In [4]:
train_path = 'train.txt'
val_path = 'val.txt'
test_path = 'test.txt'
category_path = 'item_category.txt'

In [10]:
historical_dict = {}
cat_dict={}
dic = {}
with open(category_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        item = int(line[0])
        category = int(line[1])            
        dic[item] = category
with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]
           cat_dict[user]=[dic[item]]
with open(val_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]
           cat_dict[user]=[dic[item]]
with open(test_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        line = line.strip().split(',')
        user = int(line[0])
        item = int(line[1])
        if user in historical_dict:
           historical_dict[user].append(item)
           cat_dict[user].append(dic[item])
        else:
           historical_dict[user] = [item]     
           cat_dict[user]=[dic[item]]    

100%|██████████| 813098/813098 [00:00<00:00, 1035173.52it/s]


In [11]:
print(cat_dict[0])
print(historical_dict[0])

[0, 1, 4, 5, 4, 5, 1, 7, 5, 8, 11, 5, 5, 5, 5, 12, 6, 6, 6, 9, 10, 2, 3, 2, 2, 10]
[0, 1, 5, 6, 7, 8, 10, 14, 15, 16, 19, 20, 21, 22, 23, 25, 9, 12, 13, 17, 24, 2, 3, 4, 11, 18]


In [63]:
train={}
test={}
val={}

for i in historical_dict.keys():
    train[i]=[-1]
    test[i]=[-1]
    val[i]=[-1]
    unique_elements = list(set(cat_dict[i]))
    element_counts = [cat_dict[i].count(element) for element in unique_elements]
    element_indices = {}
    for index, element in enumerate(cat_dict[i]):
        if element in element_indices:
            element_indices[element].append(index)
        else:
            element_indices[element] = [index]
    
    for j in unique_elements:
        my_list=element_indices[j]
        if(len(my_list)<=3):
            for k in my_list:
                rand=random.random()
                if rand<0.6:
                    train[i].append(historical_dict[i][k])
                elif rand>0.6 and rand<0.8:
                    test[i].append(historical_dict[i][k])
                else:
                    val[i].append(historical_dict[i][k])
        else:
            result_list = [historical_dict[i][index] for index in my_list]
            train_sp,temp_sp,_,_=train_test_split(result_list, result_list, test_size=0.4, random_state=42)
            train[i]+=train_sp
            test_sp,val_sp,_,_=train_test_split(temp_sp, temp_sp, test_size=0.5, random_state=42)
            test[i]+=test_sp
            val[i]+=val_sp

In [64]:
print(historical_dict[i])
print(cat_dict[i])
print(unique_elements)
print(element_counts)
print(element_indices)
print(train[i])
print(test[i])
print(val[i])

[17405, 12116, 29694, 88278, 93464, 57600, 134994, 123631, 103594, 79643, 115140, 108404]
[182, 467, 467, 965, 265, 4, 908, 965, 392, 324, 322, 322]
[322, 4, 965, 324, 392, 265, 908, 467, 182]
[2, 1, 2, 1, 1, 1, 1, 2, 1]
{182: [0], 467: [1, 2], 965: [3, 7], 265: [4], 4: [5], 908: [6], 392: [8], 324: [9], 322: [10, 11]}
[-1, 108404, 88278, 123631, 93464, 134994, 29694]
[-1, 12116, 17405]
[-1, 115140, 57600, 79643, 103594]


In [67]:
file_path = "train_v2.txt"
train_num=0
test_num=0
val_num=0
with open(file_path, "w") as file:
    for user in train.keys():
        for item in train[user]:
            if item==-1:
                continue
            else:
                train_num+=1
                file.write(str(user)+','+str(item)+"\n")
file_path = "test_v2.txt"
with open(file_path, "w") as file:
    for user in test.keys():
        for item in train[user]:
            if item==-1:
                continue
            else:
                test_num+=1
                file.write(str(user)+','+str(item)+"\n")
file_path = "val_v2.txt"
with open(file_path, "w") as file:
    for user in val.keys():
        for item in train[user]:
            if item==-1:
                continue
            else:
                val_num+=1
                file.write(str(user)+','+str(item)+"\n")
        